In [5]:
using Flux,Random,Optimisers,Statistics

mutable struct GlobalModel
    subnets
end

function GlobalModel()
    model1=Chain(Dense(3=>3,bias=false,init=rand))
    model2=Chain(Dense(3=>1,bias=false,init=rand))
    subnets=[model1,model2]
    GlobalModel(subnets)
end

function call_train(glob::GlobalModel,inputs)
    return glob.subnets[2](glob.subnets[1](inputs[1]))
end

(glob::GlobalModel)(inputs) =call_train(glob,inputs)
Flux.@functor GlobalModel 

function loss(y_terminal,inputs)
    delta = (y_terminal.-sum(inputs[1],dims=1)).^2
    return mean(delta)
end

function train!(glob::GlobalModel)
    #opt = Optimisers.Adam(0.01)
    #opt_state = Optimisers.setup(opt, glob)
    optim = Flux.setup(Flux.Adam(0.001), glob)
    
    my_log = []
    for epoch in 1:2000
        input=[randn((3,5)),randn((2,5))]
        val, grads = Flux.withgradient(glob) do m
        result = m(input)
        loss(result, input)
        end

        if epoch%500==0
            inp=[randn((3,5)),randn((2,5))]
            println("Epoch ",epoch, " losses ",loss(glob(inp),inp))
        end
        Flux.update!(optim, glob, grads[1])
        println(Flux.params(glob))
    end
end


glob=GlobalModel();
train!(glob)

Params([[0.07430217557320051 0.9689724272874729 0.3798133696067019; 0.2888782609467666 0.7753667550443928 0.2943443818225528; 0.013855368128869148 0.39480679246871125 0.07230796405109981], [0.4604974593687646 0.5152468258242517 0.6470263899732658]])
Params([[0.07530314251022335 0.9680465953347873 0.38058801585301427; 0.28987928322474493 0.7744414189552719 0.2951189064678167; 0.014856343750472872 0.3938810358642101 0.07308259178967423], [0.461193844625911 0.5162147001838987 0.6460375500426419]])
Params([[0.07609437664120679 0.9671943891275686 0.3812816588732404; 0.29067059994798133 0.7735894960985982 0.2958124884470718; 0.015647537548264957 0.3930292014180029 0.07377596350796564], [0.4617182533992334 0.5169672501923478 0.6450636088518482]])
Params([[0.07690057455363745 0.9668063050702537 0.3820520179977765; 0.29147713641226963 0.7732019666404851 0.2965828749421616; 0.016453142548458616 0.3926396926053782 0.07454553518395692], [0.4624334797256463 0.5178141312930288 0.645352939471637]])
P